## Build Knowledge Graph from Amazon Data

In [2]:
import os
import sys
import json
import re
import random
import copy
from tqdm import tqdm

In [3]:
data_dir = "/recsys_data/RecSys/recommenders/tests/resources/deeprec/slirec"
filename = 'meta_Electronics.json'

In [4]:
all_keys = set()
all_vals = []
entity_dict = dict()
category_dict = dict()
count = 0
with open(os.path.join(data_dir, filename), 'r') as fr:
    for line in fr.readlines():
        jdict = eval(line)

#         jstr = line.strip()
#         jstr = re.sub(r"[0-9]\"", "", jstr)
#         jstr = re.sub(r"[^:]\s\"\w|\w\" ", " ", jstr)    
#         jstr = jstr.replace("\'", "\"")
#         jdict = json.loads(jstr)
        keys = jdict.keys()
        all_keys.update(keys)
        all_vals.append(jdict)
        entity_dict[jdict['asin']] = {'id': count}
        if 'related' in jdict:
            entity_dict[jdict['asin']]['related'] = jdict['related']
        for cat in jdict['categories'][0]:
            if cat not in category_dict:
                category_dict[cat] = []
            category_dict[cat].append(jdict['asin'])
#         entity_dict[jdict['asin']]['categories'] = jdict['categories']
        count += 1
print(count)

498196


In [4]:
# for cat in category_dict:
#     ratio = len(category_dict[cat])/count
#     if ratio > 0.1:
#         print(cat, len(category_dict[cat]), ratio)
        
# for cat in category_dict:
#     if 10 < len(category_dict[cat]) < 100:
#         print(cat, len(category_dict[cat]))

In [7]:
def write_all_files(out_dir, categories, entities, max_examples=None):
    include_categories = []
    for cat in categories:
        ratio = len(categories[cat])/count
#         if ratio > cutoff and ratio < 0.8:
        if 10 < len(category_dict[cat]) < 100:
            include_categories.append(cat)
    
    relations = copy.deepcopy(include_categories)
    include_categories.extend(['also_viewed', 'buy_after_viewing'])
    print(f"Added {len(include_categories)} relations")
        
    # entity2id.txt
    with open(os.path.join(out_dir, 'AE', 'entity2id.txt'), 'w') as fw:
        fw.write(str(len(entities)) + '\n')
        for e in entities:
            fw.write(e + '\t' + str(entities[e]['id']) + '\n')
    
    # relation2id.txt
    with open(os.path.join(out_dir, 'AE', 'relation2id.txt'), 'w') as fw:
        fw.write(str(len(include_categories)) + '\n')
        for ii, r in enumerate(include_categories):
            fw.write(r + '\t' + str(ii) + '\n')
    
    # train2id.txt
    train, valid, test = [], [], []
    count_train, count_valid, count_test = 0, 0, 0

    for jj, cat in enumerate(relations):
        similar_entities = categories[cat]
        print(cat, len(similar_entities))
        for e1 in similar_entities:
            for e2 in similar_entities:
                if e1 != e2:
                    tuples = (entities[e1]['id'], entities[e2]['id'], jj)
                    rnd = random.random()
                    if rnd <= 0.7:
                        train.append(tuples)
                        count_train += 1
                    elif rnd <= 0.85:
                        valid.append(tuples)
                        count_valid += 1
                    else:
                        test.append(tuples)
                        count_test += 1
    
    print(f"Train: {count_train}, Validation: {count_valid} and Test: {count_test}")
    
    for e in entities:
        if 'related' in entities[e]:
            if 'also_viewed' in entities[e]['related']:
                connected = entities[e]['related']['also_viewed']
                jj = include_categories.index('also_viewed')
                for e2 in connected:
                    if e2 in entities:
                        tuples = (entities[e]['id'], entities[e2]['id'], jj)
                        rnd = random.random()
                        if rnd <= 0.7:
                            train.append(tuples)
                            count_train += 1
                        elif rnd <= 0.85:
                            valid.append(tuples)
                            count_valid += 1
                        else:
                            test.append(tuples)
                            count_test += 1
                            
            if 'buy_after_viewing' in entities[e]['related']:
                connected = entities[e]['related']['buy_after_viewing']
                jj = include_categories.index('buy_after_viewing')
                for e2 in connected:
                    if e2 in entities:
                        tuples = (entities[e]['id'], entities[e2]['id'], jj)
                        rnd = random.random()
                        if rnd <= 0.7:
                            train.append(tuples)
                            count_train += 1
                        elif rnd <= 0.85:
                            valid.append(tuples)
                            count_valid += 1
                        else:
                            test.append(tuples)
                            count_test += 1
    
    print(f"Train: {count_train}, Validation: {count_valid} and Test: {count_test}")
    
    random.shuffle(train)
    random.shuffle(valid)
    random.shuffle(test)
    
    if max_examples:
        train = train[:max_examples]
        valid = valid[:max_examples]
        test = test[:max_examples]
    
    with open(os.path.join(out_dir, 'AE', 'train2id.txt'), 'w') as fw:
        fw.write(str(len(train)) + '\n')
        for tup in train:
            out = [str(e) for e in tup]
            fw.write(' '.join(out) + '\n')
        
    with open(os.path.join(out_dir, 'AE', 'valid2id.txt'), 'w') as fw:
        fw.write(str(len(valid)) + '\n')
        for tup in valid:
            out = [str(e) for e in tup]
            fw.write(' '.join(out) + '\n')
        
    with open(os.path.join(out_dir, 'AE', 'test2id.txt'), 'w') as fw:
        fw.write(str(len(test)) + '\n')
        for tup in test:
            out = [str(e) for e in tup]
            fw.write(' '.join(out) + '\n')
    


In [8]:
write_all_files(out_dir='/recsys_data/RecSys/OpenKE/benchmarks/',
                categories=category_dict,
                entities=entity_dict,
                max_examples=10000
               )

Added 219 relations
Stylus Pens 63
USB Fans 91
VHS 72
Hi-8 45
Color Correction & Compensation Filters 94
APS Cameras 59
VHS-C 23
Systems 24
Wireless Jack Systems 25
VCR Rewinders 32
Caller ID Displays 89
Cable Straps 64
PS/2 Cables 50
SCSI Cables 57
Zip Discs 45
Minidisc Players 51
External Zip Drives 52
Internal Modems 84
Tape Decks 57
Minidiscs 32
Security Locks 72
Automotive 44
CD Recorders 55
Tuners 95
CD-RW Discs 77
SmartMedia Cards 36
Radar Systems 14
Changers 46
Handheld CB Radios 11
Catadioptric 84
Barlow Lenses 29
Telescope Cases 36
Photo Adapters 27
Tape Libraries 19
Multimedia Cards 48
Parallel Port Cards 59
DVD-RAM Discs 53
Bluetooth Headsets 99
Holsters & Clips 24
VTR Decks 17
Cassette Storage 21
Projectors 36
Slide Projectors 35
Single-Use Cameras 99
Presentation Supplies 32
Binding Machine Supplies 13
Calculator Accessories 93
Tabletop & Travel Tripods 22
TV Turntables 44
Port Tubes 22
Dew Caps 37
Fixed-mount CB Radios 74
Binocular Accessories 56
Clock Radios 61
Mounting

In [127]:
'0545016266' in entity_dict

False

In [85]:
entity_dict['BT008T2BGK']

{'id': 498195,
 'related': {'also_viewed': ['B0007T27H8', 'B00425S1H8', 'B000BI95W0'],
  'buy_after_viewing': ['B0007T27H8', 'B00425S1H8']}}

In [8]:
desc_dict = dict()
count_missing = 0
with open(os.path.join(data_dir, filename), 'r') as fr:
    for line in fr.readlines():
        jdict = eval(line)
        if 'description' not in jdict:
            count_missing += 1
        else:
            desc_dict[jdict['asin']] = jdict['description']
print(count_missing)

38726


In [9]:
img_dict = dict()
count_missing = 0
with open(os.path.join(data_dir, filename), 'r') as fr:
    for line in fr.readlines():
        jdict = eval(line)
        if 'imUrl' not in jdict:
            count_missing += 1
        else:
            img_dict[jdict['asin']] = jdict['imUrl']
print(count_missing)

175


In [7]:
all_keys

{'asin',
 'brand',
 'categories',
 'description',
 'imUrl',
 'price',
 'related',
 'salesRank',
 'title'}

In [47]:
import re
ss = 'Size: 2.875"(W) x 8.5"(L) x 1.125"(D)'
# ss.replace(r"[0-9]\"", '')
re.sub(r"[0-9]\"", "", ss)

#
ss = '"key": "HDTV" accented by a subtle "nook" logo'
re.sub(r"[^:]\s\"\w|\w\" ", " ", ss)

'Size: 2.87(W) x 8.(L) x 1.12(D)'

In [6]:
from collections import defaultdict

In [9]:
def data_process_with_time(fname, pname, sep="\t", file_write=False):
    User = defaultdict(list)
    Items = set()
    user_dict, item_dict = {}, {}

    with open(fname, 'r') as fr:
        for line in fr:
            u, i, t = line.rstrip().split(sep)
            User[u].append((i, t))
            Items.add(i)
    
    print(len(User), len(Items))
    item_count = 1
    for item in Items:
        item_dict[item] = item_count
        item_count += 1

    count_del = 0
    user_count = 1
    if file_write:
        with open(pname, 'w') as fw:
            for user in User.keys():
                if len(User[user]) <= 2:
                    del User[user]
                    count_del += 1
                else:
                    # user_dict[user] = user_count
                    items = sorted(User[user], key=lambda x: x[1])
                    items = [item_dict[x[0]] for x in items]
                    for item in items:
                        fw.write(str(user_count) + ' ' + str(item) + '\n')
                    user_count += 1
    else:
        for user in User.keys():
            if len(User[user]) <= 2:
                # del User[user]
                count_del += 1
            else:
                user_dict[user] = user_count
                user_count += 1
        
    print(user_count-1, count_del)
    return user_dict, item_dict

In [11]:
udict, idict = data_process_with_time("/recsys_data/RecSys/SASRec-tf2/data/ae_original.txt", "")

63161 85930
63114 47


In [12]:
len(udict), len(idict)

(63114, 85930)

In [15]:
inv_udict = {v:k for k, v in udict.items()}
inv_idict = {v:k for k, v in idict.items()}

In [16]:
inv_idict[1]

'B0013J5XZE'

In [17]:
entity_dict['B0013J5XZE']

{'id': 95536,
 'related': {'also_bought': ['B000XB9GV0',
   'B003HFB9B4',
   'B003U8YAWI',
   'B0028ZCXQ4',
   'B000XB9GUG',
   'B00494P6AW',
   'B006LZQ6O2',
   'B000XB4412',
   'B001ENW61I',
   'B00009UTL1',
   '032171105X',
   'B004BFZHO4',
   'B003HF3A34',
   'B003UOIMAS',
   'B00452V288',
   'B001M4HXB2',
   'B0009BX0AM',
   'B0028ZETAC',
   'B0083Z6R4W',
   'B00009UTZA',
   'B001OKBLEE',
   'B004BFXBXI',
   'B001HAER88',
   'B00DNADZME',
   'B007FH1KX2',
   'B000CLNHXY',
   'B003OU51LG',
   'B003GR6IUK',
   'B0090BSSZO',
   'B0007DDK7A',
   'B00BXA7N6A',
   'B002XUN2AA',
   'B003N8SKDU',
   'B007FH1LJU',
   'B002YWHSVG',
   'B003VBO4U2',
   'B004YG7JXW',
   'B003TYDBYQ',
   'B001DIG46M',
   'B003CRZFT4',
   'B006ISEJ9Q',
   '0321832752',
   'B00HO0BYQS',
   'B001E4LC6C',
   '0321580141',
   'B005GMWEI8',
   'B001M4HTAW',
   'B002Z3H36O',
   'B004CBTCFC',
   'B004YG7JV4',
   'B00AA478UQ',
   'B0034WR19Y',
   'B00009UTL9',
   'B0079M711S',
   'B001TANZ0W',
   'B00270VQTK',
   'B00B

In [18]:
all_keys

{'asin',
 'brand',
 'categories',
 'description',
 'imUrl',
 'price',
 'related',
 'salesRank',
 'title'}

## Find Items with Title & Descriptions

In [21]:
ddict = {}
with open(os.path.join(data_dir, filename), 'r') as fr:
    for line in fr.readlines():
        jdict = eval(line)
        if jdict['asin'] in idict:
            ddict[jdict['asin']] = {}
            if 'title' in jdict.keys():
                ddict[jdict['asin']]['title'] = jdict['title']
            if 'description' in jdict.keys():
                ddict[jdict['asin']]['description'] = jdict['description']

print(len(ddict))

85930


In [22]:
ddict['B0013J5XZE']

{'title': 'Honl Photo Professional Color Correction Gel Filter Kit for Photo Speed System',
 'description': 'Use Honl Color Correction filters to color correct your strobes for tungsten, fluorescent or cool daylight lighting environments. Use CTO filters to match your flash output to tungsten lights, or simply to give a pleasing warm tone to your portraits. Use Full + Green to match your flash output to fluorescent lights. These superior filters are pre-cut with velcro strips attached so they can be quickly fastened to a Speed Strap (sold separately). Honl Filters can be used in conjunction with Honl Snoots, Gobos and Grids. The usable filter area is 2.5" x 4" and can be trimmed to fit individual strobes'}

In [23]:
count = 0
for item in ddict:
    if len(ddict[item]) == 0:
        count += 1
print(count)

155


In [32]:
def nwords(txt, n):
    return ' '.join(txt.split()[:n])

In [33]:
keep_words = 100
max_len = 0
with open('/recsys_data/RecSys/SASRec-tf2/data/ae_item_description.txt', 'w') as fw:
    for item_number in range(1, 85931):
        item_name = inv_idict[item_number]
        temp_dict = ddict[item_name]
        title, desc = '', ''
        if 'title' in temp_dict:
            title = temp_dict['title'].replace('\n', '')
        if 'description' in temp_dict:
            desc = temp_dict['description'].replace('\n', '')
        otxt = title + ' ' + desc
        if len(otxt.split()) > max_len:
            max_len = len(otxt.split())
        otxt = nwords(otxt, keep_words)
        fw.write(otxt + '\n')
print(max_len)

18060


## Find Items with Images

In [34]:
imgdict = {}
with open(os.path.join(data_dir, filename), 'r') as fr:
    for line in fr.readlines():
        jdict = eval(line)
        if jdict['asin'] in idict:
            imgdict[jdict['asin']] = {}
            if 'imUrl' in jdict.keys():
                imgdict[jdict['asin']]['imUrl'] = jdict['imUrl']

print(len(imgdict))

85930


In [35]:
# Items with no image information
count = 0
for item in imgdict:
    if len(imgdict[item]) == 0:
        count += 1
print(count)

58


In [36]:
imgdict['B0013J5XZE']

{'imUrl': 'http://ecx.images-amazon.com/images/I/2135RN5gz3L.jpg'}

In [43]:
from os.path import basename
import urllib

item_name = 'B0013J5XZE'
imgUrl = imgdict[item_name]['imUrl']
imgData = urllib.request.urlopen(imgUrl).read()
fileName = item_name + '_' + imgUrl.split('/')[-1]
print(filename)
output = open(fileName,'wb')
output.write(imgData)
output.close()